In [1]:
import pandas as pd
from tqdm import tqdm
import sys
sys.path.append("..")
from src.scraping_funcs import *

Definimos una lista de supermercados y se generan las URLs correspondientes para acceder a la información en la página de FACUA.

In [2]:
supermercados = ["mercadona", "carrefour", "eroski", "dia", "hipercor", "alcampo"]

In [3]:
superurls = [f'https://super.facua.org/{s}/' for s in supermercados]

Ahora para el scraping realizamos una especie de función Matrioska, donde vamos a ir yendo por niveles, extrayendo por capas los links que se irán usando en los procesos posteriores hasta dar con las tablas.

In [4]:
dfs_super_list = []  # Lista donde iremos almacenando todos los registros de los productos de todos los supermercados
supermercados = ["mercadona", "carrefour", "eroski", "dia", "hipercor", "alcampo"]  # Lista de nombres de supermercados
# Generamos URLs para cada supermercado
superurls = [f'https://super.facua.org/{s}/' for s in supermercados]

# Iteramos sobre la lista de supermercados
for i in tqdm(range(len(supermercados))):
    dfs_cat_list = []  # Lista para almacenar los DataFrames de las categorías de cada supermercado
    
    # Obtenemos las URLs y nombres de las categorías para el supermercado actual
    urls_cats, cats = get_cats(superurls[i])
    
    # Iteramos sobre las URLs de categorías y sus nombres
    for url2, cat in zip(urls_cats, cats):
        dfs_subcat_list = []  # Lista para almacenar los DataFrames de las subcategorías
        
        # Obtenemos las URLs y nombres de las subcategorías para la categoría actual
        urls_subcat, subcats = get_cats(url2)
        
        # Verificamos si no hay subcategorías
        if not urls_subcat and not subcats:  # Condición que se aplica para el caso de que no existan subcategorías
            urls_hist = get_hist(url2)  # Obtenemos las URLs históricas de productos
            dfs_hist_list = []  # Lista para almacenar DataFrames de productos históricos
            
            # Iteramos sobre las URLs históricas
            for url3 in urls_hist:
                df_producto = get_tab(url3)  # Obtenemos la tabla de productos de la URL
                dfs_hist_list.append(df_producto)  # Añadimos el DataFrame a la lista
            
            df_hist = pd.concat(dfs_hist_list)  # Concatenamos todos los DataFrames de productos históricos
            df_hist["Subcategoria"] = None  # Asignamos None a la columna de subcategoría, ya que no hay subcategorías
            dfs_subcat_list.append(df_hist)  # Añadimos el DataFrame histórico a la lista de subcategorías
        else:
            # Si hay subcategorías, iteramos sobre ellas
            for url3, subcat in zip(urls_subcat, subcats):
                dfs_hist_list = []  # Lista para almacenar DataFrames de productos históricos por subcategoría
                
                urls_hist = get_hist(url3)  # Obtenemos las URLs históricas para la subcategoría
                for url4 in urls_hist:  # Iteramos sobre las URLs históricas
                    df_producto = get_tab(url4)  # Obtenemos la tabla de productos de la URL
                    dfs_hist_list.append(df_producto)  # Añadimos el DataFrame a la lista
                
                df_hist = pd.concat(dfs_hist_list)  # Concatenamos todos los DataFrames de productos históricos
                df_hist["Subcategoria"] = subcat  # Asignamos el nombre de la subcategoría
                dfs_subcat_list.append(df_hist)  # Añadimos el DataFrame a la lista de subcategorías

        # Concatenamos todos los DataFrames de subcategorías para la categoría actual
        df_subcat = pd.concat(dfs_subcat_list)
        df_subcat["Categoria"] = cat  # Asignamos el nombre de la categoría
        dfs_cat_list.append(df_subcat)  # Añadimos el DataFrame de subcategorías a la lista de categorías
    
    # Concatenamos todos los DataFrames de categorías para el supermercado actual
    df_cat = pd.concat(dfs_cat_list)
    df_cat["Supermercado"] = supermercados[i]  # Asignamos el nombre del supermercado
    dfs_super_list.append(df_cat)  # Añadimos el DataFrame de categorías a la lista de supermercados


100%|██████████| 6/6 [09:42<00:00, 97.06s/it] 


In [5]:
df_complete = pd.concat(dfs_super_list)

Ahora procedemos con la limpieza del DataFrame.

In [6]:
df_complete["Precio (€)"] = df_complete["Precio (€)"].str.replace(",",".").astype(float) # Cambiamos a decimalpoint y pasamos a float

In [7]:
# Sustituimos = por 0, cambiamos a decimalpoint y eliminamos paréntesis. Rellenamos nulos con 0 y expandimos.
df_complete = pd.concat([df_complete,df_complete["Variación"].str.replace("=","0").str.replace(",",".").str.replace("(","").str.replace(")","").str.replace("%","").str.split(expand=True).fillna("0")], axis=1)

In [8]:
# Renombramos las nuevas columnas
df_complete = df_complete.rename({0: 'abs_variacion', 1: 'precent_variacion'}, axis=1).drop(columns='Variación')

In [9]:
df_complete.columns

Index(['Día', 'Precio (€)', 'Nombre', 'Subcategoria', 'Categoria',
       'Supermercado', 'abs_variacion', 'precent_variacion'],
      dtype='object')

Reordenamos las columnas 

In [10]:
columnas = df_complete.columns
orden = [2, 0, 1, -2, -1, 3, 4, 5]
columnas_reordenadas = [columnas[i] for i in orden]

In [11]:
df_complete = df_complete.reindex(columns=columnas_reordenadas).reset_index(drop=True)

Convertimos cantidades

In [12]:
df_complete["abs_variacion"] = df_complete["abs_variacion"].astype(float)
df_complete["precent_variacion"] = df_complete["precent_variacion"].astype(float)

In [13]:
df_complete["Día"] = df_complete["Día"].map(lambda x: pd.to_datetime(x, format='%d/%m/%Y'))

Extraemos las cantidades de los nombres de los productos

In [14]:
df_complete[["Cantidad", "Unidad"]] = df_complete["Nombre"].apply(extraer_cantidad).tolist()

Por último corregimos el signo de las variaciones percentuales

In [15]:
df_complete.loc[df_complete["abs_variacion"] < 0, "precent_variacion"] *= -1

Guardamos en un csv para uso posterior

In [16]:
df_complete.to_csv('../datos/df_complete.csv')